<a href="https://colab.research.google.com/github/NicoPatalagua/BigData/blob/master/Spark_Dataframes_Windows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Taller Spark-Dataframes-Windows**
## *Nicolás Patalagua*
## Infraestructura para BigData - Universidad Sergio Arboleda
# **Spark**
[Apache Spark](https://spark.apache.org/) es un framework de computación en clúster open-source. Spark fue desarrollado en sus inicios por Matei Zaharia en la Universidad de California,el AMPLab de la UC Berkeley en 2009. Fue liberado como código abierto en 2010 bajo licencia BSD. El código base del proyecto Spark fue donado más tarde a la Apache Software Foundation que se encarga de su mantenimiento desde entonces. 

## **¿Para que sirve Spark?**

*Spark proporciona una interfaz para la programación de clusters completos con Paralelismo de Datos implícito y tolerancia a fallos.Apache Spark se puede considerar un sistema de computación en clúster de propósito general y orientado a la velocidad. Proporciona APIs en Java, Scala, Python y R. También proporciona un motor optimizado que soporta la ejecución de grafos en general. También soporta un conjunto extenso y rico de herramientas de alto nivel entre las que se incluyen Spark SQL (para el procesamiento de datos estructurados basada en SQL), MLlib para implementar machine learning, GraphX para el procesamiento de grafos y Spark Streaming.*

# **Funciones de ventana en Spark**
*Las funciones de ventana permiten a los usuarios de Spark SQL calcular resultados como el rango de una fila determinada o un promedio móvil en un rango de filas de entrada. Mejoran significativamente la expresividad de las API de SQL y DataFrame de Spark.
En esencia, una función de ventana calcula un valor de retorno para cada fila de entrada de una tabla en función de un grupo de filas, llamado Marco . Cada fila de entrada puede tener un marco único asociado. Esta característica de las funciones de ventana las hace más potentes que otras funciones y permite a los usuarios expresar varias tareas de procesamiento de datos que son difíciles (si no imposibles) de expresar de manera concisa sin las funciones de ventana.* 

http://queirozf.com/entries/spark-dataframe-examples-window-functions

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark
import os
import time 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

from google.colab import drive
drive.mount('/content/gdrive')
from pyspark import SparkConf, SparkContext
conf = SparkConf().setAppName("app")
sc = SparkContext.getOrCreate();

     |████████████████████████████████| 217.8MB 59kB/s 
     |████████████████████████████████| 204kB 35.8MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=f7e1a13e85cc0dd55e734da5f342f5c7451da9091b0093d3426a8a507498b076
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Importamos las bibliotecas y dependencias que necesitamos
import numpy as np
import pandas as pd
from pyspark.sql import Window
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
sp= SparkSession.builder.appName('test').getOrCreate()

# ***Taller***
**1. Descargar las acciones de Ecopetrol, Avianca, y Grupo Aval para el último año de Yahoo finances(23 de sept-2018 y 23 de sept-2019).** 

*Los CSV de acciones se obtienen de [Yahho! Finance](https://finance.yahoo.com/), el cual es un servicio de Yahoo! que proporciona información financiera y comentarios con un enfoque en los mercados de los Estados Unidos.*



Las acciones de [Ecopetrol](www.ecopetrol.com.co) se pueden consultar en [EC-Yahoo! Finance](https://finance.yahoo.com/quote/EC?p=EC&.tsrc=fin-srch), y pertenecen a la primera compañía de petróleo de Colombia, antiguamente llamada Empresa Colombiana de Petróleos S.A.Es la cuarta empresa petrolera más grande de Latinoamérica.

Las acciones de [Avianca S.A](www.avianca.com) se pueden consultar en [AVH-Yahoo! Finance](https://finance.yahoo.com/quote/AVH?p=AVH&.tsrc=fin-srch), y pertenecen a Aerovías del Continente Americano, anteriormente Aerovías Nacionales de Colombia. Es la mayor aerolínea de Colombia. Fundada en 1919 con el nombre de SCADTA, es mundialmente la segunda aerolínea más antigua después de la KLM que fue fundada 58 días antes y la más antigua del mundo con operaciones ininterrumpidas.

Las acciones del [Grupo Aval](www.grupoaval.com) se pueden consultar en [AVA-Yahoo! Finance](https://finance.yahoo.com/quote/AVAL?p=AVAL&.tsrc=fin-srch), corresponden a las acciones del Grupo Acciones y Valores S.A, un n conglomerado empresarial colombiano dedicado a una amplia variedad de actividades, principalmente financieras. El Grupo AVAL es controlado por Luis Carlos Sarmiento Angulo.

Una vez descargados son agregados a un repositorio de Github y posteriormente cargados en nuestro entorno de trabajo:


*   [EC](https://github.com/NicoPatalagua/BigData/blob/master/EC.csv)
*   [AVH](https://github.com/NicoPatalagua/BigData/blob/master/AVH.csv)
*   [AVA](https://github.com/NicoPatalagua/BigData/blob/master/AVA.csv)



In [0]:
#Asignamos el dataset (CSV) a una variable
ObjEC = sp.read.option("inferSchema","true").csv("/content/EC.csv",header=True)
ObjAVH = sp.read.option("inferSchema","true").csv("/content/AVH.csv",header=True)
ObjAVA = sp.read.option("inferSchema","true").csv("/content/AVA.csv",header=True)

Imprimir cada esquema de los dataset:

In [47]:
ObjEC

DataFrame[Date: timestamp, Open: double, High: double, Low: double, Close: double, Adj Close: double, Volume: int]

In [48]:
ObjAVH

DataFrame[Date: timestamp, Open: double, High: double, Low: double, Close: double, Adj Close: double, Volume: int]

In [49]:
ObjAVA

DataFrame[Date: timestamp, Open: double, High: double, Low: double, Close: double, Adj Close: double, Volume: int, Year: string, Month: string, Day: string]

El resultado que nos envia, significa que los dataset no estan corectamente esquematizados y se debe eparar la fecha por año, mes y día.

Por tanto se debe realizar un ajuste de esquema a cada uno de los dataset. 

Para EC:

In [0]:
ObjSplt= F.split(ObjEC['Date'], '-')
ObjEC = ObjEC.withColumn('Year', ObjSplt.getItem(0))
ObjEC = ObjEC.withColumn('Month', ObjSplt.getItem(1))
ObjEC = ObjEC.withColumn('Day', ObjSplt.getItem(2))

Verificamos que el Esquema se modificó:

In [58]:
ObjEC.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)



Efectivamente la fecha se dividio en año, mes y día.


Para AVH:

In [0]:
ObjSplt= F.split(ObjAVH['Date'], '-')
ObjAVH = ObjAVH.withColumn('Year', ObjSplt.getItem(0))
ObjAVH = ObjAVH.withColumn('Month', ObjSplt.getItem(1))
ObjAVH = ObjAVH.withColumn('Day', ObjSplt.getItem(2))

Verificamos que el Esquema se modificó:

In [53]:
ObjAVH.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)



Efectivamente la fecha se dividio en año, mes y día.

Para AVA:

In [0]:
ObjSplt= F.split(ObjAVA['Date'], '-')
ObjAVA = ObjAVA.withColumn('Year', ObjSplt.getItem(0))
ObjAVA = ObjAVA.withColumn('Month', ObjSplt.getItem(1))
ObjAVA = ObjAVA.withColumn('Day', ObjSplt.getItem(2))

Verificamos que el Esquema se modificó:

In [55]:
ObjAVA.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)



Efectivamente la fecha se dividio en año, mes y día.

**2.Porcentaje de retorno diaro.**

Para EC:


In [67]:
ObjOpen = ObjEC.groupby('Year','Month','Day').agg(F.sum('Open').alias('Open'))
ObjClose = ObjEC.groupby('Year','Month','Day').agg(F.sum('Close').alias('Close'))
ObjJoin = ObjOpen.join(ObjClose, ['Year','Month','Day'])
ObjPRDEC = ObjJoin.withColumn('PR ECOPETROL', (((ObjJoin['Close'] - ObjJoin['Open']) / ObjJoin['Open'] )* 100))
ObjPRDEC.orderBy('Year','Month','Day')
ObjPRDEC.show()

+----+-----+-----------+---------+---------+--------------------+
|Year|Month|        Day|     Open|    Close|        PR ECOPETROL|
+----+-----+-----------+---------+---------+--------------------+
|2019|   09|20 00:00:00|17.690001|    17.57| -0.6783549644796429|
|2018|   10|23 00:00:00|25.110001|    24.74| -1.4735204510744624|
|2019|   05|01 00:00:00|    18.48|    18.07| -2.2186147186147194|
|2019|   01|03 00:00:00|16.559999|     16.5|-0.36231282381116836|
|2018|   10|26 00:00:00|23.120001|    23.59|   2.032867559132032|
|2019|   01|09 00:00:00|     18.6|    18.76|  0.8602150537634415|
|2019|   09|18 00:00:00|    17.66|17.690001| 0.16988108720271014|
|2019|   06|03 00:00:00|16.540001|    16.82|  1.6928596316288018|
|2019|   07|16 00:00:00|     19.0|18.870001| -0.6842052631579029|
|2019|   08|19 00:00:00|    16.34|16.110001| -1.4075826193390417|
|2018|   09|25 00:00:00|26.209999|    26.32|  0.4196909736623815|
|2018|   10|11 00:00:00|25.309999|    25.15| -0.6321572750753671|
|2019|   0

Para AVH:

In [68]:
ObjOpen = ObjAVH.groupby('Year','Month','Day').agg(F.sum('Open').alias('Open'))
ObjClose = ObjAVH.groupby('Year','Month','Day').agg(F.sum('Close').alias('Close'))
ObjJoin = ObjOpen.join(ObjClose, ['Year','Month','Day'])
ObjPRDAVH = ObjJoin.withColumn('PR AVIANCA', (((ObjJoin['Close'] - ObjJoin['Open']) / ObjJoin['Open'] )* 100))
ObjPRDAVH.orderBy('Year','Month','Day')
ObjPRDAVH.show()

+----+-----+-----------+----+-----+--------------------+
|Year|Month|        Day|Open|Close|          PR AVIANCA|
+----+-----+-----------+----+-----+--------------------+
|2019|   09|20 00:00:00|3.05| 2.99| -1.9672131147540857|
|2018|   10|23 00:00:00|5.35|  5.4|  0.9345794392523498|
|2019|   05|01 00:00:00|3.73| 3.85|  3.2171581769437028|
|2019|   01|03 00:00:00|4.15| 4.13|-0.48192771084338454|
|2018|   10|26 00:00:00|5.25| 5.26|  0.1904761904761864|
|2019|   01|09 00:00:00|4.58| 4.65|  1.5283842794759888|
|2019|   09|18 00:00:00|3.13| 3.14|  0.3194888178913812|
|2019|   06|03 00:00:00|3.67| 3.46|  -5.722070844686647|
|2019|   07|16 00:00:00| 4.3| 4.25| -1.1627906976744147|
|2019|   08|19 00:00:00|3.65| 3.52| -3.5616438356164357|
|2018|   09|25 00:00:00|5.74|  5.7| -0.6968641114982584|
|2018|   10|11 00:00:00|5.42| 5.32| -1.8450184501844953|
|2019|   07|03 00:00:00|3.69| 3.89|   5.420054200542011|
|2019|   08|28 00:00:00|2.73| 2.58|  -5.494505494505491|
|2018|   11|20 00:00:00|5.26| 5

Para AVA:

In [71]:
ObjOpen = ObjAVA.groupby('Year','Month','Day').agg(F.sum('Open').alias('Open'))
ObjClose = ObjAVA.groupby('Year','Month','Day').agg(F.sum('Close').alias('Close'))
ObjJoin = ObjOpen.join(ObjClose, ['Year','Month','Day'])
ObjPRDAVA = ObjJoin.withColumn('PR GRUPO AVAL', (((ObjJoin['Close'] - ObjJoin['Open']) / ObjJoin['Open'] )* 100))
ObjPRDAVA.orderBy('Year','Month','Day')
ObjPRDAVA.show()

+----+-----+-----------+---------+---------+--------------------+
|Year|Month|        Day|     Open|    Close|       PR GRUPO AVAL|
+----+-----+-----------+---------+---------+--------------------+
|2019|   09|20 00:00:00|47.619999|47.880001|  0.5459932916000273|
|2018|   10|23 00:00:00|51.849998|51.599998|-0.48216009574388025|
|2019|   05|01 00:00:00|43.139999|    42.27| -2.0166875757229383|
|2019|   01|03 00:00:00|    42.73|41.970001| -1.7786075356891962|
|2018|   10|26 00:00:00|51.599998|    51.48| -0.2325542725796278|
|2019|   01|09 00:00:00|41.580002|41.610001| 0.07214766367735277|
|2019|   09|18 00:00:00|47.349998|47.560001|  0.4435121623447595|
|2019|   06|03 00:00:00|41.919998|42.630001|  1.6937095273716385|
|2019|   07|16 00:00:00|    45.16|    45.57|  0.9078830823737903|
|2019|   08|19 00:00:00|    46.59|46.349998| -0.5151362953423568|
|2018|   09|25 00:00:00|50.400002|    50.59|  0.3769801437706347|
|2018|   10|11 00:00:00|51.099998|50.880001|-0.43052252174256317|
|2019|   0

**3.Porcentaje de retorno Mensual.**

Para EC:


In [72]:
ObjPRMEC= ObjPRDEC.groupby('Year','Month').agg(F.avg('PR ECOPETROL').alias('PR ECOPETROL'))
ObjPRMEC=ObjPRMEC.orderBy('Year','Month')
ObjPRMEC.show()

+----+-----+--------------------+
|Year|Month|        PR ECOPETROL|
+----+-----+--------------------+
|2018|   09|  1.3604574942012235|
|2018|   10| -0.6449597997062496|
|2018|   11| -0.8896312516430294|
|2018|   12| -0.9766423294363785|
|2019|   01|  0.8345324145987951|
|2019|   02| 0.35187492019506605|
|2019|   03| 0.23044138448293458|
|2019|   04|  -0.713712232460116|
|2019|   05|  -0.081363679607359|
|2019|   06| 0.17839549977884622|
|2019|   07|-0.30583588648860416|
|2019|   08|-0.37583497541655986|
|2019|   09| 0.36662064695531954|
+----+-----+--------------------+



Para AVH:

In [73]:
ObjPRMAVH= ObjPRDAVH.groupby('Year','Month').agg(F.avg('PR AVIANCA').alias('PR AVIANCA'))
ObjPRMAVH=ObjPRMAVH.orderBy('Year','Month')
ObjPRMAVH.show()

+----+-----+--------------------+
|Year|Month|          PR AVIANCA|
+----+-----+--------------------+
|2018|   09| -0.5188698108072496|
|2018|   10| -0.3658693544499807|
|2018|   11|-0.14912911543472887|
|2018|   12| -1.3011778973839219|
|2019|   01|  0.5716433302845226|
|2019|   02|-0.35210257785344323|
|2019|   03| -0.8737195524510399|
|2019|   04| -0.5850313008407249|
|2019|   05| 0.17274173243499125|
|2019|   06|  0.3330837786005799|
|2019|   07|  0.5473780887446394|
|2019|   08| -1.8596725468647934|
|2019|   09|  0.9843850485986011|
+----+-----+--------------------+



Para AVA:

In [74]:
ObjPRMAVA= ObjPRDAVA.groupby('Year','Month').agg(F.avg('PR GRUPO AVAL').alias('PR GRUPO AVAL'))
ObjPRMAVA=ObjPRMAVA.orderBy('Year','Month')
ObjPRMAVA.show()

+----+-----+--------------------+
|Year|Month|       PR GRUPO AVAL|
+----+-----+--------------------+
|2018|   09|-0.06298583534760246|
|2018|   10|0.047343298849797655|
|2018|   11|0.022506843550737104|
|2018|   12|-0.35003723144228255|
|2019|   01|-0.04241029085526...|
|2019|   02|-0.03692076578349763|
|2019|   03|-0.02737067258607...|
|2019|   04|  0.2682942266447532|
|2019|   05|-0.06407148498604325|
|2019|   06| 0.28422729112121803|
|2019|   07| 0.11565505505324412|
|2019|   08| 0.06892819257512892|
|2019|   09| 0.16912663241720144|
+----+-----+--------------------+



**4.Mes con mayor porcentaje de retorno.**


Para EC:

In [77]:
ObjPRMEC.orderBy(col('PR ECOPETROL').desc()).show(1)

+----+-----+------------------+
|Year|Month|      PR ECOPETROL|
+----+-----+------------------+
|2018|   09|1.3604574942012235|
+----+-----+------------------+
only showing top 1 row



Para AVH:

In [78]:
ObjPRMAVH.orderBy(col('PR AVIANCA').desc()).show(1)

+----+-----+------------------+
|Year|Month|        PR AVIANCA|
+----+-----+------------------+
|2019|   09|0.9843850485986011|
+----+-----+------------------+
only showing top 1 row



Para AVA:

In [79]:
ObjPRMAVA.orderBy(col('PR GRUPO AVAL').desc()).show(1)

+----+-----+-------------------+
|Year|Month|      PR GRUPO AVAL|
+----+-----+-------------------+
|2019|   06|0.28422729112121803|
+----+-----+-------------------+
only showing top 1 row



**5.Hallar y gráficar la media móvil de 20 dias para Ecopetrol.**


La grafica se obtine con el siguiente código:

**6.Graficar comportamiento del precio de cierre para las 3 acciones a lo largo del año.**


Se realiza una gráfica donde se obtenga el comportamiento solicitado de las acciones de las tres empresas.

**7.Calcular y graficar franjas Bollinger para K=2.**


Franjas Bollinger para EC:

Franjas Bollinger para AVH:

Franjas Bollinger para AVA:

**8.Para qué fechas la tendecia se sale de las franjas calculadas en el punto anterior?**

**9.Calcular y graficar las franjas Bollinger con K=2 y K=1.**

Franjas Bollinger para EC con K=1:

Franjas Bollinger para EC con K=2:

Franjas Bollinger para AVH con K=1:

Franjas Bollinger para AVH con K=2:

Franjas Bollinger para AVA con K=1:

Franjas Bollinger para AVA con K=2:

**10.Determinar para qué fechas es conveniente comprar usando la técnica de doble franja Bollinger.**

Para EC:

Para AVH:

Para AVA:

**11.Determinar para qué fechas es conveniente vender en corto la técnica de doble franja Bollinger.**

Para EC:

Para AVH:

Para AVA: